# Extract-Transform-Load from Kiva GraphQL API

Authors: Hye Lee, Adam Miner, Charles Ramey

In [42]:
#imports

import pandas as pd
import requests as re
import json
import time
import queue
from concurrent.futures import ThreadPoolExecutor

In [41]:
#connect to Kiva API
base_url = 'https://api.kivaws.org/graphql'

graphql_query = """
query GetData($off: Int!) {
  lend {
    loans(filters: {status: all}, sortBy: newest, limit: 100, offset: $off) {
      values {
        activity {
          id
        }
        borrowerCount
        borrowers {
          id
          borrowedAmount
          gender
          isPrimary
        }
        descriptionInOriginalLanguage
        disbursalDate
        fundraisingDate
        geocode {
          city
          country {
              name
              isoCode
              region
              ppp
              numLoansFundraising
              fundsLentInCountry
          }
          state
          postalCode
          latitude
          longitude
        }
        id
        lenderRepaymentTerm
        raisedDate
        sector {
          id
        }
        status
        tags
        terms {
          currency
          currencyFullName
          disbursalAmount
          disbursalDate
          loanAmount
          lenderRepaymentTerm
        }
      }
    }
  }
}
"""

In [6]:
big_json = []

In [44]:
#how to programmatically pull the newest data. The example is supposed to pull the million most recent, but last time I ran it I got 700 records
#Can I get more data using a nested loop? 
start_time = time.time()

with ThreadPoolExecutor(max_workers= 6) as executor:
    executor._work_queue=queue.Queue(maxsize=0)
    for i in range(1000):
        res = re.post(
        base_url,
            json = {
                    'query': graphql_query,
                    'variables': {'off': i * 100}
                }
        )
        res = res.json()
        data_json = dict(res)
        spec_json_data = data_json['data']['lend']['loans']['values']
        big_json.extend(spec_json_data)

time.time() - start_time

6512.468266963959

In [ ]:
#tqdm library to wrap function in status bar

In [5]:
#testing out data ingestion on the sample file given
# sample = dict(pd.read_json('.\data\sample.json'))
# with open('.\data\sample.json', 'r') as f:
#     json_data = f.read()
# sample = json.loads(json_data)
# sample_data = sample['data']['lend']['loans']['values']
# activity_data = sample_data[1]

In [6]:
# res = res.json()

In [7]:
# data_json = dict(res)

In [8]:
# spec_json_data = data_json['data']['lend']['loans']['values']

In [9]:
# activity_data = spec_json_data[1]

In [10]:
# print(activity_data)

In [45]:
#The data is in list form of dictionaries. Go through each item in the list, and pull out specific columns into their own lists to put into dataframes
#Country codes
big_list = []
countryName = []
isoCode = []
region = []
ppp = []
numLoansFundraising = []
fundsLentinCountry = []

#trying this method out: this is to break out the list into easier parsable parts
for i in big_json:
    big_list.append(i['geocode']['country'])
    
    
for i in big_list:    
    countryName.append(i['name'])
    isoCode.append(i['isoCode'])
    region.append(i['region'])
    ppp.append(i['ppp'])
    numLoansFundraising.append(i['numLoansFundraising'])
    fundsLentinCountry.append(i['fundsLentInCountry'])

In [46]:
country_df = pd.DataFrame({'countryName': countryName, 'isoCode': isoCode, 'region': region, 'ppp': ppp,
                       'numLoansFundraising': numLoansFundraising, 'fundsLentinCountry': fundsLentinCountry})

In [47]:
#remove duplicates from country dataframe
country_df = country_df.drop_duplicates()

In [48]:
export_country_df = country_df.to_csv('.\data\country_ref.csv', index = False)

In [49]:
#Geocode
big_list = []
# city_id = [] #to get a unique pkey for city
city = []
state = []
isoCode = []
postalCode = []
latitude = []
longitude = []
# city_num = 0

#trying this method out: this is to break out the list into easier parsable parts
for i in big_json:
    big_list.append(i['geocode']['country'])
#     city_id.append(city_num)
    city.append(i['geocode']['city'])
    state.append(i['geocode']['state'])
    postalCode.append(i['geocode']['postalCode'])
    latitude.append(i['geocode']['latitude'])
    longitude.append(i['geocode']['longitude'])
#     city_num += 1
    
for i in big_list:    
    isoCode.append(i['isoCode'])

In [50]:
#this one has city id
# geocode_df = pd.DataFrame({'city_id': city_id, 'city': city, 'isocode': isocode, 'state': state,
#                        'postalCode': postalCode, 'latitude': latitude, 'longitude': longitude})

# This one does not have city id
geocode_df = pd.DataFrame({'city': city, 'state': state, 'isoCode': isoCode,
                       'postalCode': postalCode, 'latitude': latitude, 'longitude': longitude})

In [51]:
#remove duplicates from geocode dataframe
geocode_df = geocode_df.drop_duplicates(subset = ['city', 'isoCode', 'state', 'postalCode', 'latitude', 'longitude'])

In [52]:
export_geocode_df = geocode_df.to_csv('.\data\geocode_ref.csv', index = False)

In [53]:
#Currencies
currency = []
currencyFullName = []

for i in big_json:
    currency.append(i['terms']['currency'])
    currencyFullName.append(i['terms']['currencyFullName'])

In [54]:
#one of the currencies has weird question marks in it. I am going to pull the question marks out
currencyFullName = [i.replace('?', '') for i in currencyFullName]

In [55]:
currencies_df = pd.DataFrame({'currency': currency, 'currencyFullName': currencyFullName})

In [56]:
#remove duplicates from currency dataframe
currencies_df = currencies_df.drop_duplicates()

In [57]:
export_currencies_df = currencies_df.to_csv('.\data\currencies_ref.csv', index = False)

In [58]:
#The data is in list form of dictionaries. Go through each item in the list, and pull out specific columns into their own lists to put into dataframes
#loans
loanId = []
activityId = []
sectorId = []
borrowerId = []
borrowerCount = []
disbursalDate = []
disbursalAmount = []
lenderRepaymentTerm = []
status = []
loanAmount = []

for i in big_json:
    loanId.append(i['id'])
    activityId.append(i['activity']['id'])
    sectorId.append(i['sector']['id'])
    borrowerId.append(i['borrowers'][0]['id'])
    borrowerCount.append(i['borrowerCount'])
    disbursalDate.append(i['disbursalDate'])
    disbursalAmount.append(i['terms']['disbursalAmount'])
    lenderRepaymentTerm.append(i['lenderRepaymentTerm'])
    status.append(i['status'])
    loanAmount.append(i['terms']['loanAmount'])

In [59]:
# #this one has some generated id fields in it
# loan_df = pd.DataFrame({'loan_id': loan_id, 'borrowerCount': borrowerCount, 'borrowers_id': borrowers_id, 'disbursalDate': disbursalDate,
#                        'lenderRepaymentTerm': lenderRepaymentTerm, 'status': status, 'currency': currency, 'currencyFullName': currencyFullName,
#                        'disbursalAmount': disbursalAmount, 'disbursalDate': disbursalDate, 'loanAmount': loanAmount, 'isocode': isocode, 'city_id': city_id})

#This one does not have the generated id fields in them, and includes some columns to join up with tables in the Load part of ETL
loan_df = pd.DataFrame({'loanId': loanId, 'activityId': activityId, 'sectorId': sectorId, 'borrowerId': borrowerId, 'borrowerCount': borrowerCount, 'disbursalDate': disbursalDate,
                       'disbursalAmount': disbursalAmount, 'lenderRepaymentTerm': lenderRepaymentTerm, 'status': status, 'loanAmount': loanAmount, 'currencyFullName': currencyFullName,
                        'isoCode': isoCode, 'city': city})


In [ ]:
loan_df = loan_df.drop_duplicates(subset = ['loanId'])

In [60]:
export_loan_df = loan_df.to_csv('.\data\loan_df.csv', index = False)

In [61]:
#Crowdsourced
loanId = []
borrowerId = []
fundraisingDate = []
raisedDate = []

for i in big_json:
    loanId.append(i['id'])
    borrowerId.append(i['borrowers'][0]['id'])
    fundraisingDate.append(i['fundraisingDate'])
    raisedDate.append(i['raisedDate'])

In [62]:
crowdsourced_df = pd.DataFrame({'loanId': loanId, 'borrowerId': borrowerId, 'fundraisingDate': fundraisingDate, 'state': state,
                       'raisedDate': raisedDate})

In [63]:
crowdsourced_df = crowdsourced_df.drop_duplicates(subset = ['borrowerId'])

In [64]:
export_crowdsourced_df = crowdsourced_df.to_csv('.\data\crowdsourced_df.csv', index = False)

In [65]:
#Utility
loanId = []
activityId = []
sectorId = []
tags = []
descriptionInOriginalLanguage = []

#trying this method out: this is to break out the list into easier parsable parts
for i in big_json:
    loanId.append(i['id'])
    activityId.append(i['activity']['id'])
    sectorId.append(i['sector']['id'])
    tags.append(i['tags'])
    descriptionInOriginalLanguage.append(i['descriptionInOriginalLanguage'])

In [66]:
utility_df = pd.DataFrame({'loanId': loanId, 'activityId': activityId, 'sectorId': sectorId, 'tags': tags,
                       'descriptionInOriginalLanguage': descriptionInOriginalLanguage})

In [67]:
export_utility_df = utility_df.to_csv('.\\data\\utility_df.csv', index = False)

In [68]:
# #Borrowers
# loan_id = []
# borrowers_id = []
# borrowedAmount = []
# isPrimary = []

# #trying this method out: this is to break out the list into easier parsable parts
# for i in big_json:
#     loan_id.append(i['id'])
#     borrowers_id.append(i['borrowers'][0]['id'])
#     borrowedAmount.append(i['borrowers'][0]['borrowedAmount'])
#     isPrimary.append(i['borrowers'][0]['isPrimary'])

In [69]:
# borrowers_df = pd.DataFrame({'loan_id': loan_id, 'borrowers_id': borrowers_id, 'borrowedAmount': borrowedAmount, 'isPrimary': isPrimary})

In [70]:
# export_borrowers_df = borrowers_df.to_csv('.\\data\\borrowers_df.csv', index = False)

In [71]:
#Demographics
borrowerId = []
borrowedAmount = []
isPrimary = []
gender = []


for i in big_json:
    borrowerId.append(i['borrowers'][0]['id'])
    borrowedAmount.append(i['borrowers'][0]['borrowedAmount'])
    isPrimary.append(i['borrowers'][0]['isPrimary'])
    gender.append(i['borrowers'][0]['gender'])

In [72]:
demographics_df = pd.DataFrame({'borrowerId': borrowerId, 'borrowedAmount': borrowedAmount, 'isPrimary': isPrimary, 'gender': gender})

In [73]:
demographics_df = demographics_df.drop_duplicates(subset = ['borrowerId'])

In [74]:
export_demographics_df = demographics_df.to_csv('.\data\demographics_df.csv', index = False)